In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

# Recession

Main purpose is to practice data cleanings and combining datasets from different sources. So we want to create a dataframe with a number of different variables and see which of them is a better predictor of recession (if there's any). We're going to take 1950-01-01 as a starting point (it's harder to find a reliable enough data for some variables prior to that date) and fill the dataset with data for every month starting from that. For some variables there's no monthly data, only quarterly, so we'll fill that with artificial data elements.

In [2]:
import numpy as np
import pandas as pd
import re

First we want to create the dataset. Start with GDP table, then add GDP rates, then go on with the next tables.

A note on dates:

Despite that the dates are given in the format `yyyy-mm-dd`, where `dd` is 01, in fact the data is given for _that entire month_. For example, `1948-03-01` means that the data is for March 1948, **not** for Feb 1948. Maybe i'll will fix that later.

### GDP

In [3]:
df = pd.read_csv('Real_GDP_USA.csv')
df.head()
df.shape

,DATE,GDPC1
0,1947-01-01,2182.681
1,1947-04-01,2176.892
2,1947-07-01,2172.432
3,1947-10-01,2206.452
4,1948-01-01,2239.682


(309, 2)

Here we want to convert the variable into date-time object

In [4]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [5]:
df_to_append = pd.read_csv('Real_GDP_Rate_USA.csv')
df_to_append.head()
df_to_append.shape

,DATE,A191RL1Q225SBEA
0,1947-04-01,-1.0
1,1947-07-01,-0.8
2,1947-10-01,6.4
3,1948-01-01,6.2
4,1948-04-01,6.8


(308, 2)

There's a discrepancy in data, so we convert variable into date-time object and just add an initial datapoint with value 0, that's the starting point of the rate changes.

In [6]:
df_to_append['DATE'] = pd.to_datetime(df_to_append['DATE'])

In [7]:
df_to_append = pd.concat([pd.DataFrame({'DATE':[pd.to_datetime('1947-01-01')], 'A191RL1Q225SBEA': 0}), df_to_append]).reset_index(drop=True)

In [8]:
df_to_append.head()

,DATE,A191RL1Q225SBEA
0,1947-01-01,0.0
1,1947-04-01,-1.0
2,1947-07-01,-0.8
3,1947-10-01,6.4
4,1948-01-01,6.2


In [9]:
df['GDP rate'] = df_to_append['A191RL1Q225SBEA']
df.head()

,DATE,GDPC1,GDP rate
0,1947-01-01,2182.681,0.0
1,1947-04-01,2176.892,-1.0
2,1947-07-01,2172.432,-0.8
3,1947-10-01,2206.452,6.4
4,1948-01-01,2239.682,6.2


In [10]:
df.rename(columns={'GDPC1':'GDP', 'DATE': 'Date'}, inplace=True)
df.head(1)

,Date,GDP,GDP rate
0,1947-01-01,2182.681,0.0


In [11]:
# df.reset_index()
df.columns
df.info()

Index(['Date', 'GDP', 'GDP rate'], dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      309 non-null    datetime64[ns]
 1   GDP       309 non-null    float64       
 2   GDP rate  309 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 7.4 KB


To merge with the other tables we need to make it for every month, not quarter. So we'll expand the table with monthly data for every quarter filling the GDP and rate values with values for the relevant quarter.

In [12]:
new_rows = []

for _, row in df.iterrows():
    for month in range(3):
        new_date = row['Date'] + pd.DateOffset(months=month)
        new_rows.append({
            'Date': new_date,
            'GDP': row['GDP'],
            'GDP rate': row['GDP rate']
        })

new_df = pd.DataFrame(new_rows)

new_df = new_df.reset_index(drop=True)

new_df.head(15)

,Date,GDP,GDP rate
0,1947-01-01,2182.681,0.0
1,1947-02-01,2182.681,0.0
2,1947-03-01,2182.681,0.0
3,1947-04-01,2176.892,-1.0
4,1947-05-01,2176.892,-1.0
5,1947-06-01,2176.892,-1.0
6,1947-07-01,2172.432,-0.8
7,1947-08-01,2172.432,-0.8
8,1947-09-01,2172.432,-0.8
9,1947-10-01,2206.452,6.4


In [13]:
df = new_df
df.info()
df.tail(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      927 non-null    datetime64[ns]
 1   GDP       927 non-null    float64       
 2   GDP rate  927 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 21.9 KB


,Date,GDP,GDP rate
926,2024-03-01,22758.752,1.4


### Recession dates

In [14]:
df_rec = pd.read_csv('US_Rcession_dates_.csv')
df_rec.head()
df_rec.tail()

,DATE,USREC
0,1854-12-01,1
1,1855-01-01,0
2,1855-02-01,0
3,1855-03-01,0
4,1855-04-01,0


,DATE,USREC
2030,2/1/24,0
2031,3/1/24,0
2032,4/1/24,0
2033,5/1/24,0
2034,6/1/24,0


In [15]:
# part of dates is in incorrrect date format. we need to deal with that first.
# df_rec['DATE'] = pd.to_datetime(df_rec['DATE'])

In [16]:
# try to convert to datetime object
df_rec['Date_converted'] = pd.to_datetime(df_rec['DATE'], errors='coerce')

In [17]:
df_rec[df_rec['Date_converted'].isna()]

,DATE,USREC,Date_converted
541,1/1/00,1,NaT
542,2/1/00,1,NaT
543,3/1/00,1,NaT
544,4/1/00,1,NaT
545,5/1/00,1,NaT
...,...,...,...
2030,2/1/24,0,NaT
2031,3/1/24,0,NaT
2032,4/1/24,0,NaT
2033,5/1/24,0,NaT


So sstarting from 1900 the date format goes like that: m/d/yy, we need yyyy-mm-dd. And the dates look same for 19xx and 2xxx, so just converting them would cause errors like after 1999-12-01 will come 1900-01-01. So we need to differentiate them.

In [18]:
df_rec.loc[df_rec['DATE'] == '1/1/00']

df_rec.iloc[1739:1743]

,DATE,USREC,Date_converted
541,1/1/00,1,NaT
1741,1/1/00,0,NaT


,DATE,USREC,Date_converted
1739,11/1/99,0,NaT
1740,12/1/99,0,NaT
1741,1/1/00,0,NaT
1742,2/1/00,0,NaT


In [19]:
def correct_format(date_string, index):
    try:
        parts = date_string.split('/')
        if len(parts) == 3:
            month = parts[0].zfill(2)
            day = parts[1].zfill(2)
            year = parts[2].zfill(2)
            if index < 1741: # easiest way is just to make index number asa condition
                year = '19' + year
            else:
                year = '20' + year
            return f"{year}-{month}-{day}"
        return date_string
    except Exception as e:
        return date_string

In [20]:
df_rec['Date'] = [correct_format(date, idx) for idx, date in df_rec['DATE'].items()]

In [21]:
df_rec.iloc[1738:1750]

,DATE,USREC,Date_converted,Date
1738,10/1/99,0,NaT,1999-10-01
1739,11/1/99,0,NaT,1999-11-01
1740,12/1/99,0,NaT,1999-12-01
1741,1/1/00,0,NaT,2000-01-01
1742,2/1/00,0,NaT,2000-02-01
1743,3/1/00,0,NaT,2000-03-01
1744,4/1/00,0,NaT,2000-04-01
1745,5/1/00,0,NaT,2000-05-01
1746,6/1/00,0,NaT,2000-06-01
1747,7/1/00,0,NaT,2000-07-01


In [22]:
df_rec.drop(columns=['DATE', 'Date_converted'], inplace=True)

In [23]:
df_rec.rename({'USREC':'Recession'}, axis='columns', inplace=True)
df_rec.head()

,Recession,Date
0,1,1854-12-01
1,0,1855-01-01
2,0,1855-02-01
3,0,1855-03-01
4,0,1855-04-01


In [24]:
df_rec['Date'] = pd.to_datetime(df_rec['Date'])

### Unemployment

In [25]:
df_unempl = pd.read_excel('Monthly_Unemployment_Rate.xlsx', header=11)
df_unempl.head()
df_unempl.info()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    77 non-null     int64  
 1   Jan     77 non-null     float64
 2   Feb     77 non-null     float64
 3   Mar     77 non-null     float64
 4   Apr     77 non-null     float64
 5   May     77 non-null     float64
 6   Jun     77 non-null     float64
 7   Jul     76 non-null     float64
 8   Aug     76 non-null     float64
 9   Sep     76 non-null     float64
 10  Oct     76 non-null     float64
 11  Nov     76 non-null     float64
 12  Dec     76 non-null     float64
dtypes: float64(12), int64(1)
memory usage: 7.9 KB


To work with this table we need to use Year column as is and the rest columns have transposed, so that in the end we get data in the following format: `yyyy-mm-dd xx`

In [26]:
df_unempl['Year'].min()
df_unempl['Year'].max()

1948

2024

In [27]:
# df_unempl.drop(columns='Year', inplace=True)
df_test = df_unempl.iloc[:5]
df_test

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7


In [28]:
df_unempl = pd.melt(df_unempl, id_vars=['Year'], var_name="Month", value_name="Percentage")

In [29]:
# new_df.sort_values(by=['Year','Month'])
df_unempl['Date'] = df_unempl['Year'].astype('str') + '-' + df_unempl['Month'] + "-01"
# new_df['Date'] = str(new_df.Year)+'-'+str(new_df.Month)+'-01'
df_unempl['Date'] = pd.to_datetime(df_unempl['Date'])
df_unempl.drop(columns=['Year', 'Month'], inplace=True)

In [30]:
df_unempl = df_unempl.sort_values(by = ['Date'])
df_unempl.reset_index(drop=True, inplace=True)

In [31]:
df_unempl.tail(10)

,Percentage,Date
914,3.8,2024-03-01
915,3.9,2024-04-01
916,4.0,2024-05-01
917,4.1,2024-06-01
918,NaN,2024-07-01
919,NaN,2024-08-01
920,NaN,2024-09-01
921,NaN,2024-10-01
922,NaN,2024-11-01
923,NaN,2024-12-01


In [32]:
df_unempl = df_unempl.iloc[:915]

In [33]:
df_unempl = df_unempl.reset_index(drop=True)

In [34]:
df_unempl.head(1)
df_unempl.tail(1)
df_unempl.shape

,Percentage,Date
0,3.4,1948-01-01


,Percentage,Date
914,3.8,2024-03-01


(915, 2)

### Industrial production

In [35]:
df_ind = pd.read_excel('ind_prod_index.xlsx')
df_ind.head()
df_ind.tail()
df_ind.info()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1919,4.8654,4.6504,4.5160,4.5966,4.6235,4.9192,5.2149,5.2955,5.1880,5.1342,5.0536,5.1342
1,1920,5.6181,5.6181,5.5106,5.2149,5.3493,5.4030,5.2686,5.2955,5.1073,4.8923,4.4891,4.2203
2,1921,3.9784,3.8977,3.7902,3.7902,3.8977,3.8708,3.8439,3.9784,4.0052,4.2472,4.1934,4.1665
3,1922,4.3278,4.5160,4.7579,4.5966,4.8385,5.0805,5.0805,4.9729,5.2417,5.5374,5.7794,5.9406
4,1923,5.8062,5.8869,6.0751,6.2095,6.2901,6.2363,6.1826,6.0751,5.9406,5.9138,5.9138,5.7794


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
101,2020,101.3372,101.6718,97.6060,84.6812,86.0108,91.6745,95.0037,95.9294,95.8914,96.5256,96.9529,98.2039
102,2021,98.8135,95.5072,98.1929,98.3317,99.1867,99.6483,100.0668,100.0412,98.9955,100.3542,101.2684,101.1948
103,2022,101.2146,101.8458,102.6732,102.9024,102.9659,102.8224,103.0505,103.1703,103.5326,103.4442,103.1058,101.8266
104,2023,102.7476,102.8003,102.8143,103.2241,102.9809,102.3809,103.0722,103.0951,103.3081,102.5781,102.8868,102.6309
105,2024,101.4830,102.6045,102.4062,102.4329,103.3282,103.9941,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    106 non-null    int64  
 1   Jan     106 non-null    float64
 2   Feb     106 non-null    float64
 3   Mar     106 non-null    float64
 4   Apr     106 non-null    float64
 5   May     106 non-null    float64
 6   Jun     106 non-null    float64
 7   Jul     105 non-null    float64
 8   Aug     105 non-null    float64
 9   Sep     105 non-null    float64
 10  Oct     105 non-null    float64
 11  Nov     105 non-null    float64
 12  Dec     105 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 10.9 KB


In [36]:
df_ind_2 = pd.melt(df_ind, id_vars='Year', var_name='Month', value_name='IP Index')

In [37]:
df_ind_2['Date'] = df_ind_2['Year'].astype('str') + '-' + df_ind_2['Month'] + "-01"
df_ind_2['Date'] = pd.to_datetime(df_ind_2['Date'])
df_ind_2.drop(columns=['Year', 'Month'], inplace=True)

In [38]:
df_ind_2 = df_ind_2.sort_values('Date')
df_ind_2.reset_index(drop=True, inplace=True)

In [39]:
df_ind_2.dropna(inplace=True)
df_ind_2.head(20)

,IP Index,Date
0,4.8654,1919-01-01
1,4.6504,1919-02-01
2,4.5160,1919-03-01
3,4.5966,1919-04-01
4,4.6235,1919-05-01
5,4.9192,1919-06-01
6,5.2149,1919-07-01
7,5.2955,1919-08-01
8,5.1880,1919-09-01
9,5.1342,1919-10-01


### Dow Jones

In [40]:
df_dj = pd.read_excel('dja-performance-report-daily.xls', header=5)
df_dj.head()
df_dj.tail()
df_dj.info()

,Index Name,Effective Date,Close Value,TR Close Value,Net Change,Daily Volume,Open Value,Intraday High,Intraday Low,Theoretical Open,Theoretical High,Theoretical Low
0,Dow Jones Industrial Average,05/26/1896,40.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dow Jones Industrial Average,05/27/1896,40.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dow Jones Industrial Average,05/28/1896,40.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dow Jones Industrial Average,05/29/1896,40.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dow Jones Industrial Average,06/01/1896,40.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Index Name,Effective Date,Close Value,TR Close Value,Net Change,Daily Volume,Open Value,Intraday High,Intraday Low,Theoretical Open,Theoretical High,Theoretical Low
36655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36659,Source: S&P Dow Jones Indices LLC.\n\nYou may ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36660 entries, 0 to 36659
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Index Name        36656 non-null  object 
 1   Effective Date    36655 non-null  object 
 2   Close Value       36655 non-null  float64
 3   TR Close Value    11246 non-null  float64
 4   Net Change        35576 non-null  float64
 5   Daily Volume      3018 non-null   float64
 6   Open Value        3018 non-null   float64
 7   Intraday High     3018 non-null   float64
 8   Intraday Low      3018 non-null   float64
 9   Theoretical Open  2969 non-null   float64
 10  Theoretical High  2902 non-null   float64
 11  Theoretical Low   2902 non-null   float64
dtypes: float64(10), object(2)
memory usage: 3.4+ MB


In [41]:
df_dj_ = df_dj[['Effective Date', 'Close Value']]

In [42]:
df_dj_.head()

,Effective Date,Close Value
0,05/26/1896,40.94
1,05/27/1896,40.58
2,05/28/1896,40.20
3,05/29/1896,40.63
4,06/01/1896,40.60


In [43]:
df_dj_['Effective Date'].isnull().idxmax()

36655

In [44]:
df_dj_.iloc[36653:36660]

,Effective Date,Close Value
36653,07/29/2024,40539.93
36654,07/30/2024,40743.33
36655,NaN,NaN
36656,NaN,NaN
36657,NaN,NaN
36658,NaN,NaN
36659,NaN,NaN


In [45]:
df_dj_.dropna(inplace=True)

In [46]:
df_dj_['Date'] = pd.to_datetime(df_dj_['Effective Date'])
df_dj_.drop(columns='Effective Date', inplace=True)
df_dj_.head()

,Close Value,Date
0,40.94,1896-05-26
1,40.58,1896-05-27
2,40.20,1896-05-28
3,40.63,1896-05-29
4,40.60,1896-06-01


In [47]:
df_dj_['DJI'] = df_dj_.groupby(df_dj_['Date'].dt.to_period('M'))['Close Value'].transform('mean')

In [48]:
df_dj_.head()

,Close Value,Date,DJI
0,40.94,1896-05-26,40.587500
1,40.58,1896-05-27,40.587500
2,40.20,1896-05-28,40.587500
3,40.63,1896-05-29,40.587500
4,40.60,1896-06-01,39.121154


In [49]:
df_dj_.loc[df_dj_['Date'] == '1948-01-02']

,Close Value,Date,DJI
15273,181.04,1948-01-02,176.2564


In [50]:
df_dj = df_dj_.iloc[15273:]
df_dj.reset_index(drop=True, inplace=True)
df_dj.head()

,Close Value,Date,DJI
0,181.04,1948-01-02,176.2564
1,179.53,1948-01-05,176.2564
2,179.12,1948-01-06,176.2564
3,179.83,1948-01-07,176.2564
4,180.60,1948-01-08,176.2564


In [51]:
df_dj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21382 entries, 0 to 21381
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Close Value  21382 non-null  float64       
 1   Date         21382 non-null  datetime64[ns]
 2   DJI          21382 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 501.3 KB


Now we need to reduce the dataset to monthly data only.

In [52]:
df_dj['Year'] = pd.DatetimeIndex(df_dj['Date']).year
df_dj['Mont'] = pd.DatetimeIndex(df_dj['Date']).month

In [53]:
df_dj.head()

,Close Value,Date,DJI,Year,Mont
0,181.04,1948-01-02,176.2564,1948,1
1,179.53,1948-01-05,176.2564,1948,1
2,179.12,1948-01-06,176.2564,1948,1
3,179.83,1948-01-07,176.2564,1948,1
4,180.60,1948-01-08,176.2564,1948,1


In [54]:
df_dj_combined = df_dj[['DJI', 'Year', 'Mont']]
d = df_dj_combined.drop_duplicates()
d.head()

,DJI,Year,Mont
0,176.256400,1948,1
25,168.465909,1948,2
47,169.942692,1948,3
73,180.050385,1948,4
99,186.380000,1948,5


In [55]:
d.reset_index(drop=True, inplace=True)
d.shape

(919, 3)

In [56]:
d = d.iloc[:915]
d.tail()

,DJI,Year,Mont
910,34704.502381,2023,11
911,36947.927500,2023,12
912,37763.954762,2024,1
913,38720.518000,2024,2
914,39105.520000,2024,3


In [57]:
d['Date'] = d['Year'].astype('str') + '-' + d['Mont'].astype('str') + '-01'
d.head()

,DJI,Year,Mont,Date
0,176.256400,1948,1,1948-1-01
1,168.465909,1948,2,1948-2-01
2,169.942692,1948,3,1948-3-01
3,180.050385,1948,4,1948-4-01
4,186.380000,1948,5,1948-5-01


In [58]:
d.drop(columns=['Year', 'Mont'], inplace=True)

In [59]:
d['DJI'] = d['DJI'].round(2)
d.shape
d.head()

(915, 2)

,DJI,Date
0,176.26,1948-1-01
1,168.47,1948-2-01
2,169.94,1948-3-01
3,180.05,1948-4-01
4,186.38,1948-5-01


### Yield curve data

In [60]:
df_yield = pd.read_csv('T-Bills_3M_Yield.csv') # in fact the data is by months
df_yield.head()

,DATE,TB3MS
0,1934-01-01,0.72
1,1934-02-01,0.62
2,1934-03-01,0.24
3,1934-04-01,0.15
4,1934-05-01,0.16


In [61]:
df.tail()
df_yield[df_yield['DATE'] == '1948-01-01']

,Date,GDP,GDP rate
922,2023-11-01,22679.255,3.4
923,2023-12-01,22679.255,3.4
924,2024-01-01,22758.752,1.4
925,2024-02-01,22758.752,1.4
926,2024-03-01,22758.752,1.4


,DATE,TB3MS
168,1948-01-01,0.97


In [62]:
df_yield = df_yield.iloc[168:]
df_yield.reset_index(drop=True, inplace=True)

In [63]:
df_yield.head()

,DATE,TB3MS
0,1948-01-01,0.97
1,1948-02-01,1.00
2,1948-03-01,1.00
3,1948-04-01,1.00
4,1948-05-01,1.00


### Housing started data

In [64]:
df_hstart = pd.read_excel('starts_cust.xlsx', sheet_name='Seasonally Adjusted', header=5, usecols=['Unnamed: 0', 'Total'])
df_hstart.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_hstart.dropna(inplace=True)
df_hstart.head()

,Date,Total
0,1959-01-01 00:00:00,1657.0
1,1959-02-01 00:00:00,1667.0
2,1959-03-01 00:00:00,1620.0
3,1959-04-01 00:00:00,1590.0
4,1959-05-01 00:00:00,1498.0


In [65]:
df_hstart.shape

df_hstart['Date'] = pd.to_datetime(df_hstart['Date'])

(786, 2)

The dataframe starts from 1959, but we need it from Jan 1948. We don't have monthly data for that, only annual (from db-hstarts.pdf from Bureau of the Census). So we'll need to create artificial monthly data from annual. Each month from 1948 to Dec 1958 will have monthly value = to annual value.

In [66]:
df_hstart_art = pd.read_excel('starts_cust.xlsx', sheet_name='Sheet1', names=['Year','Total'], header=None)
df_hstart_art.head(15)

,Year,Total
0,1948,1362
1,1949,1466
2,1950,1952
3,1951,1591
4,1952,1504
5,1953,1438
6,1954,1551
7,1955,1646
8,1956,1349
9,1957,1224


Create date range

In [67]:
df_r = pd.date_range(start='1948-01-01', end='1958-12-01', freq='MS')
df_r = pd.DataFrame(df_r)

In [68]:
df_r.rename(columns={0:'Date'}, inplace=True)
df_r.head()

,Date
0,1948-01-01
1,1948-02-01
2,1948-03-01
3,1948-04-01
4,1948-05-01


In [69]:
df_r['Year'] = df_r['Date'].dt.year
df_hstart_art = df_hstart_art.merge(df_r, on='Year', how='left')

In [70]:
df_hstart_art.drop(columns='Year', inplace=True)
df_hstart_art.head()

,Total,Date
0,1362,1948-01-01
1,1362,1948-02-01
2,1362,1948-03-01
3,1362,1948-04-01
4,1362,1948-05-01


In [71]:
df_hstart_combines = pd.concat([df_hstart_art, df_hstart]).sort_values(by='Date').reset_index(drop=True)

In [72]:
df_hstart_combines = df_hstart_combines.iloc[:915]

In [73]:
df_hstart_combines.info()
df_hstart_combines.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Total   915 non-null    float64       
 1   Date    915 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 14.4 KB


,Total,Date
910,1510.0,2023-11-01
911,1568.0,2023-12-01
912,1376.0,2024-01-01
913,1546.0,2024-02-01
914,1299.0,2024-03-01


### House price index

In [74]:
df_hpi = pd.read_excel('House_Price_Index_Fig3-1_1.xls', sheet_name='Data', usecols=['Nominal Home Price Index', 'Normal Date'], header=6)
df_hpi.head()

,Nominal Home Price Index,Normal Date
0,3.557296,NaT
1,3.209111,NaT
2,3.267142,NaT
3,3.406416,NaT
4,3.969316,NaT


In [75]:
df_hpi[df_hpi['Normal Date'] == '1953-01-01']
df_hpi = df_hpi.iloc[63:]
df_hpi.head()

,Nominal Home Price Index,Normal Date
63,14.26056,1953-01-01


,Nominal Home Price Index,Normal Date
63,14.260560,1953-01-01
64,14.260560,1953-02-01
65,14.260560,1953-03-01
66,14.293686,1953-04-01
67,14.310249,1953-05-01


Same as with Housing starts, there's no monthly data for the period from Jan 1948 to Mar 1953. So we need to create it the same way.

In [76]:
df_hpi_48_53 = pd.read_excel('House_Price_Index_Fig3-1_1.xls', sheet_name='Data', usecols=['Nominal Home Price Index', 'Date'], header=6)

In [77]:
df_hpi_48_53['Date'] = df_hpi_48_53['Date'].astype(int)
df_hpi_48_53[df_hpi_48_53['Date']==1948]
df_hpi_48_53[df_hpi_48_53['Date']==1953]
# the indeces are 58:63
df_hpi_48_53.head()

,Date,Nominal Home Price Index
58,1948,11.211582


,Date,Nominal Home Price Index
63,1953,14.260560
64,1953,14.260560
65,1953,14.260560
66,1953,14.293686
67,1953,14.310249
68,1953,14.310249
69,1953,14.343374
70,1953,14.359937
71,1953,14.376500
72,1953,14.376500


,Date,Nominal Home Price Index
0,1890,3.557296
1,1891,3.209111
2,1892,3.267142
3,1893,3.406416
4,1894,3.969316


In [78]:
df_hpi_48_53 = df_hpi_48_53.iloc[58:64]

In [79]:
df_hpi_48_53_art = pd.date_range(start='1948-01-01', end='1952-12-01', freq='MS')
df_hpi_48_53_art = pd.DataFrame(df_hpi_48_53_art)
df_hpi_48_53_art.head()

,0
0,1948-01-01
1,1948-02-01
2,1948-03-01
3,1948-04-01
4,1948-05-01


In [80]:
df_hpi_48_53_art['Year'] = df_hpi_48_53_art[0].dt.year

In [81]:
df_hpi_48_53['Year'] = df_hpi_48_53['Date']

In [82]:
df_hpi_48_53_art = df_hpi_48_53_art.merge(df_hpi_48_53, on='Year', how='left')

In [83]:
df_hpi_48_53_art.shape
df_hpi_48_53_art.head()
df_hpi_48_53_art.tail()

(60, 4)

,0,Year,Date,Nominal Home Price Index
0,1948-01-01,1948,1948,11.211582
1,1948-02-01,1948,1948,11.211582
2,1948-03-01,1948,1948,11.211582
3,1948-04-01,1948,1948,11.211582
4,1948-05-01,1948,1948,11.211582


,0,Year,Date,Nominal Home Price Index
55,1952-08-01,1952,1952,12.876954
56,1952-09-01,1952,1952,12.876954
57,1952-10-01,1952,1952,12.876954
58,1952-11-01,1952,1952,12.876954
59,1952-12-01,1952,1952,12.876954


In [84]:
df_hpi_48_53_art.drop(columns=['Year', 'Date'], inplace=True)

In [85]:
df_hpi_48_53_art.rename(columns = {0:'Normal Date'}, inplace=True)
df_hpi_48_53_art.head()

,Normal Date,Nominal Home Price Index
0,1948-01-01,11.211582
1,1948-02-01,11.211582
2,1948-03-01,11.211582
3,1948-04-01,11.211582
4,1948-05-01,11.211582


In [86]:
df_hpi_combine = pd.concat([df_hpi, df_hpi_48_53_art]).sort_values(by='Normal Date').reset_index(drop=True)

In [87]:
df_hpi_combine.shape
df_hpi_combine.head()
df_hpi_combine.tail()

(913, 2)

,Nominal Home Price Index,Normal Date
0,11.211582,1948-01-01
1,11.211582,1948-02-01
2,11.211582,1948-03-01
3,11.211582,1948-04-01
4,11.211582,1948-05-01


,Nominal Home Price Index,Normal Date
908,312.417,2023-09-01
909,312.953,2023-10-01
910,NaN,2023-11-01
911,NaN,2023-12-01
912,NaN,2024-01-01


In [88]:
df_hpi_combine.dropna(inplace=True)

In [89]:
add_data = {
    'Normal Date': ['2023-11-01','2023-12-01','2024-01-01','2024-02-01','2024-03-01'],
    'Nominal Home Price Index': [284.8839432, 284.2869918, 285.4357485, 288.5099601, 292.3175519]
}

df_add_data = pd.DataFrame(add_data)
df_hpi_combine = pd.concat([df_hpi_combine, df_add_data], ignore_index=True)

In [90]:
df_hpi_combine.shape
df_hpi_combine.head()
df_hpi_combine.tail()

(915, 2)

,Nominal Home Price Index,Normal Date
0,11.211582,1948-01-01 00:00:00
1,11.211582,1948-02-01 00:00:00
2,11.211582,1948-03-01 00:00:00
3,11.211582,1948-04-01 00:00:00
4,11.211582,1948-05-01 00:00:00


,Nominal Home Price Index,Normal Date
910,284.883943,2023-11-01
911,284.286992,2023-12-01
912,285.435748,2024-01-01
913,288.509960,2024-02-01
914,292.317552,2024-03-01


### Credit data 1: Consumer loans

Number of consumer loans, not SA

In [91]:
df_cloans = pd.read_csv('CONSUMER.csv')
df_cloans.head()
df_cloans.tail()

,DATE,CONSUMER
0,1947-01-01,4.1612
1,1947-02-01,4.2966
2,1947-03-01,4.4306
3,1947-04-01,4.5687
4,1947-05-01,4.7087


,DATE,CONSUMER
925,2024-02-01,1907.4073
926,2024-03-01,1914.5808
927,2024-04-01,1912.1212
928,2024-05-01,1914.6744
929,2024-06-01,1913.9266


In [92]:
df_cloans = df_cloans.iloc[12:927]
df_cloans.reset_index(drop=True, inplace=True)

### Credit data 2: Real estates loans

Number of mortgage loans

In [93]:
df_reloans = pd.read_csv('REALLN.csv')
df_reloans.head()
df_reloans.tail()

,DATE,REALLN
0,1947-01-01,6.7960
1,1947-02-01,6.9551
2,1947-03-01,7.1249
3,1947-04-01,7.2966
4,1947-05-01,7.4699


,DATE,REALLN
925,2024-02-01,5567.0833
926,2024-03-01,5585.3950
927,2024-04-01,5597.4799
928,2024-05-01,5598.1468
929,2024-06-01,5593.3485


In [94]:
df_reloans = df_reloans.iloc[12:927]
df_reloans.reset_index(drop=True, inplace=True)

In [95]:
df_reloans.shape

(915, 2)

### Consumer price index data

Consumer Price Index for All Urban Consumers: All Items in U.S. City Average 

In [96]:
df_cpiu = pd.read_excel('Consumer_price_index.xlsx', sheet_name='Sheet2')
df_cpiu.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1913,9.8,9.8,9.8,9.8,9.7,9.8,9.9,9.9,10.0,10.0,10.1,10.0
1,1914,10.0,9.9,9.9,9.8,9.9,9.9,10.0,10.2,10.2,10.1,10.2,10.1
2,1915,10.1,10.0,9.9,10.0,10.1,10.1,10.1,10.1,10.1,10.2,10.3,10.3
3,1916,10.4,10.4,10.5,10.6,10.7,10.8,10.8,10.9,11.1,11.3,11.5,11.6
4,1917,11.7,12.0,12.0,12.6,12.8,13.0,12.8,13.0,13.3,13.5,13.5,13.7


In [97]:
df_cpiu[df_cpiu['Year']==1948]
df_cpiu = df_cpiu.iloc[35:]
df_cpiu.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
35,1948,23.7,23.5,23.4,23.8,23.9,24.1,24.4,24.5,24.5,24.4,24.2,24.1


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
35,1948,23.7,23.5,23.4,23.8,23.9,24.1,24.4,24.5,24.5,24.4,24.2,24.1
36,1949,24.0,23.8,23.8,23.9,23.8,23.9,23.7,23.8,23.9,23.7,23.8,23.6
37,1950,23.5,23.5,23.6,23.6,23.7,23.8,24.1,24.3,24.4,24.6,24.7,25.0
38,1951,25.4,25.7,25.8,25.8,25.9,25.9,25.9,25.9,26.1,26.2,26.4,26.5
39,1952,26.5,26.3,26.3,26.4,26.4,26.5,26.7,26.7,26.7,26.7,26.7,26.7


In [98]:
df_cpiu_2 = pd.melt(df_cpiu, id_vars='Year', var_name='Month', value_name='Inflation Index')
df_cpiu_2.head()

,Year,Month,Inflation Index
0,1948,Jan,23.7
1,1949,Jan,24.0
2,1950,Jan,23.5
3,1951,Jan,25.4
4,1952,Jan,26.5


In [99]:
df_cpiu_2['Date'] = df_cpiu_2['Year'].astype('str') + '-' + df_cpiu_2['Month'].astype('str') + "-01"

In [100]:
df_cpiu_2['Date'] = pd.to_datetime(df_cpiu_2['Date'])

In [101]:
df_cpiu_2.drop(columns=['Year', 'Month'], inplace=True)

In [102]:
df_cpiu_2 = df_cpiu_2.sort_values('Date').reset_index(drop=True)

In [103]:
df_cpiu_2 = df_cpiu_2.iloc[:915]

In [104]:
df_cpiu_2.shape
df_cpiu_2.head()
df_cpiu_2.tail()

(915, 2)

,Inflation Index,Date
0,23.7,1948-01-01
1,23.5,1948-02-01
2,23.4,1948-03-01
3,23.8,1948-04-01
4,23.9,1948-05-01


,Inflation Index,Date
910,307.051,2023-11-01
911,306.746,2023-12-01
912,308.417,2024-01-01
913,310.326,2024-02-01
914,312.332,2024-03-01


### Inflation rates

In [105]:
df_inf = pd.read_excel('Inflation_rates.xlsx')
df_inf.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,2024,0.03091,0.03153,0.03477,0.03357,0.03269,0.02971,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,0.06410,0.06036,0.04985,0.04930,0.04048,0.02969,0.03178,0.03665,0.03700,0.03241,0.03137,0.03352,0.04116
2,2022,0.07480,0.07871,0.08542,0.08259,0.08582,0.09060,0.08525,0.08263,0.08202,0.07745,0.07110,0.06454,0.08003
3,2021,0.01400,0.01676,0.02620,0.04160,0.04993,0.05391,0.05365,0.05251,0.05390,0.06222,0.06809,0.07036,0.04698
4,2020,0.02487,0.02335,0.01539,0.00329,0.00118,0.00646,0.00986,0.01310,0.01371,0.01182,0.01175,0.01362,0.01234


In [106]:
df_inf.drop(columns='Annual', inplace=True)

In [107]:
df_inf[df_inf['Year'] == 1948]
df_inf = df_inf.iloc[:77]
df_inf.tail()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
76,1948,0.10233,0.09302,0.06849,0.08676,0.09132,0.09545,0.0991,0.08889,0.06522,0.06087,0.04762,0.02991


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
72,1952,0.04331,0.02335,0.01938,0.02326,0.01931,0.02317,0.03089,0.03089,0.02299,0.01908,0.01136,0.00755
73,1951,0.08085,0.09362,0.09322,0.09322,0.09283,0.08824,0.07469,0.06584,0.06967,0.06504,0.06883,0.06000
74,1950,-0.02083,-0.01261,-0.00840,-0.01255,-0.00420,-0.00418,0.01688,0.02101,0.02092,0.03797,0.03782,0.05932
75,1949,0.01266,0.01277,0.01709,0.00420,-0.00418,-0.00830,-0.02869,-0.02857,-0.02449,-0.02869,-0.01653,-0.02075
76,1948,0.10233,0.09302,0.06849,0.08676,0.09132,0.09545,0.09910,0.08889,0.06522,0.06087,0.04762,0.02991


In [108]:
df_inf = df_inf.sort_values('Year')

In [109]:
df_inf_2 = pd.melt(df_inf, id_vars='Year', var_name='Month', value_name='Inflation Index')

In [110]:
df_inf_2['Date'] = df_inf_2['Year'].astype('str') + '-' + df_inf_2['Month'].astype('str') + "-01"

In [111]:
df_inf_2['Date'] = pd.to_datetime(df_inf_2['Date'])

In [112]:
df_inf_2.drop(columns=['Year', 'Month'], inplace=True)

In [113]:
df_inf_2 = df_inf_2.sort_values('Date').reset_index(drop=True)

In [114]:
df_inf_2 = df_inf_2.iloc[:915]

In [115]:
df_inf_2.shape
df_inf_2.head()
df_inf_2.tail()

(915, 2)

,Inflation Index,Date
0,0.10233,1948-01-01
1,0.09302,1948-02-01
2,0.06849,1948-03-01
3,0.08676,1948-04-01
4,0.09132,1948-05-01


,Inflation Index,Date
910,0.03137,2023-11-01
911,0.03352,2023-12-01
912,0.03091,2024-01-01
913,0.03153,2024-02-01
914,0.03477,2024-03-01


### Global economic growth

In [116]:
df_global = pd.read_excel('world_gdp_population.xlsx', usecols=['Year', 'GDP growth'])
df_global.head()

,Year,GDP growth
0,2022,0.0308
1,2021,0.0602
2,2020,-0.0307
3,2019,0.0259
4,2018,0.0329


In [117]:
add_data = {
    'Year':[2023, 2024],
    'GDP growth': [0.0280, 0.0300]
}

In [118]:
df_add_data = pd.DataFrame(add_data)

In [119]:
df_global = pd.concat([df_global, df_add_data], ignore_index=True)

In [120]:
df_global.reset_index(drop=True)
df_global = df_global.sort_values(by='Year')
df_global.head()
df_global.tail()

,Year,GDP growth
0,2022,0.0308
1,2021,0.0602
2,2020,-0.0307
3,2019,0.0259
4,2018,0.0329
...,...,...
59,1963,0.0518
60,1962,0.0531
61,1961,0.0379
62,2023,0.0280


,Year,GDP growth
61,1961,0.0379
60,1962,0.0531
59,1963,0.0518
58,1964,0.0656
57,1965,0.0555


,Year,GDP growth
2,2020,-0.0307
1,2021,0.0602
0,2022,0.0308
62,2023,0.0280
63,2024,0.0300


Add data from 1948 to 1960

In [121]:
gdp_48_60 = pd.DataFrame({'Year': range(1948, 1961), 'GDP growth': [0.0484] * (1960 - 1948 + 1)})
gdp_48_60.head()

,Year,GDP growth
0,1948,0.0484
1,1949,0.0484
2,1950,0.0484
3,1951,0.0484
4,1952,0.0484


In [122]:
df_global = pd.concat([gdp_48_60, df_global], ignore_index=True)

Transform into monthly data

In [123]:
df_globa_2 = pd.DataFrame({'Date': pd.date_range(start=f"{df_global['Year'].min()}-01-01", end=f"{df_global['Year'].max()}-12-01", freq='MS')})

In [124]:
df_globa_2['Year'] = df_globa_2['Date'].dt.year
df_globa_2 = df_globa_2.merge(df_global, on='Year', how='left')

In [125]:
df_globa_2 = df_globa_2.drop(columns=['Year'])

In [126]:
df_globa_2 = df_globa_2.iloc[:915]

In [127]:
df_globa_2.shape
df_globa_2.head()
df_globa_2.tail()

(915, 2)

,Date,GDP growth
0,1948-01-01,0.0484
1,1948-02-01,0.0484
2,1948-03-01,0.0484
3,1948-04-01,0.0484
4,1948-05-01,0.0484


,Date,GDP growth
910,2023-11-01,0.028
911,2023-12-01,0.028
912,2024-01-01,0.030
913,2024-02-01,0.030
914,2024-03-01,0.030


### S&P500

In [128]:
df_snp = pd.read_html('https://www.multpl.com/s-p-500-historical-prices/table/by-month')[0]

In [129]:
df_snp.shape
df_snp.head()
df_snp.tail()

(1844, 2)

,Date,Value
0,"Aug 27, 2024",5627.75
1,"Jul 1, 2024",5553.15
2,"Jun 1, 2024",5451.14
3,"May 1, 2024",5235.22
4,"Apr 1, 2024",5112.49


,Date,Value
1839,"May 1, 1871",4.86
1840,"Apr 1, 1871",4.74
1841,"Mar 1, 1871",4.61
1842,"Feb 1, 1871",4.50
1843,"Jan 1, 1871",4.44


In [130]:
df_snp = df_snp.sort_index(ascending=False).reset_index(drop=True)

In [131]:
df_snp['Date'] = pd.to_datetime(df_snp['Date'])

In [132]:
df_snp.loc[df_snp['Date'] == '1948-01-01']
df_snp.loc[df_snp['Date'] == '2024-03-01']

,Date,Value
924,1948-01-01,14.83


,Date,Value
1838,2024-03-01,5170.57


In [133]:
df_snp500 = df_snp.iloc[924:1839].reset_index(drop=True)

In [134]:
df_snp500.head()

,Date,Value
0,1948-01-01,14.83
1,1948-02-01,14.10
2,1948-03-01,14.30
3,1948-04-01,15.40
4,1948-05-01,16.15


### Merge tables

Merge Recession data

In [135]:
df_rec.loc[df_rec['Date'] == '1947-01-01']
df_to_append = df_rec.iloc[1105:2032]
df_to_append = df_to_append.reset_index()
df_to_append.head(1)
df_to_append.tail(1)

,Recession,Date
1105,0,1947-01-01


,index,Recession,Date
0,1105,0,1947-01-01


,index,Recession,Date
926,2031,0,2024-03-01


In [136]:
df['Recession dates'] = df_to_append['Recession']
df.head()

,Date,GDP,GDP rate,Recession dates
0,1947-01-01,2182.681,0.0,0
1,1947-02-01,2182.681,0.0,0
2,1947-03-01,2182.681,0.0,0
3,1947-04-01,2176.892,-1.0,0
4,1947-05-01,2176.892,-1.0,0


Merge Unemployment data. It starts with Jan 1948, so our final dataset becomes 1 year shorter.

In [137]:
df_copy = df.copy()

In [138]:
df = df.iloc[12:]
df = df.reset_index(drop=True)

In [139]:
df.head(1)
df.tail(1)
df.shape

,Date,GDP,GDP rate,Recession dates
0,1948-01-01,2239.682,6.2,0


,Date,GDP,GDP rate,Recession dates
914,2024-03-01,22758.752,1.4,0


(915, 4)

In [140]:
df['Unemployment'] = df_unempl['Percentage']
df.head()
df.tail()
df.shape

,Date,GDP,GDP rate,Recession dates,Unemployment
0,1948-01-01,2239.682,6.2,0,3.4
1,1948-02-01,2239.682,6.2,0,3.8
2,1948-03-01,2239.682,6.2,0,4.0
3,1948-04-01,2276.690,6.8,0,3.9
4,1948-05-01,2276.690,6.8,0,3.5


,Date,GDP,GDP rate,Recession dates,Unemployment
910,2023-11-01,22679.255,3.4,0,3.7
911,2023-12-01,22679.255,3.4,0,3.7
912,2024-01-01,22758.752,1.4,0,3.7
913,2024-02-01,22758.752,1.4,0,3.9
914,2024-03-01,22758.752,1.4,0,3.8


(915, 5)

Merge Industrial production index

In [141]:
df_ind_2.loc[df_ind_2['Date'] == '1948-01-01']

,IP Index,Date
348,14.3274,1948-01-01


In [142]:
df_ind_2.iloc[348:-3]

,IP Index,Date
348,14.3274,1948-01-01
349,14.3543,1948-02-01
350,14.1931,1948-03-01
351,14.2199,1948-04-01
352,14.4618,1948-05-01
...,...,...
1258,102.8868,2023-11-01
1259,102.6309,2023-12-01
1260,101.4830,2024-01-01
1261,102.6045,2024-02-01


In [143]:
df_copy = df.copy()

In [144]:
df['Ind.Prod. index'] = df_ind_2['IP Index']

In [145]:
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index
0,1948-01-01,2239.682,6.2,0,3.4,4.8654
1,1948-02-01,2239.682,6.2,0,3.8,4.6504
2,1948-03-01,2239.682,6.2,0,4.0,4.5160
3,1948-04-01,2276.690,6.8,0,3.9,4.5966
4,1948-05-01,2276.690,6.8,0,3.5,4.6235


Merge Dow Jones

In [146]:
df_copy = df.copy()

In [147]:
df['DJI'] = d['DJI']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38


Yield curve data

In [148]:
df_copy = df.copy()

In [149]:
df['T-bill'] = df_yield['TB3MS']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00


Merge the Housing started data

In [150]:
df_copy = df.copy()

In [151]:
df['Housing starts'] = df_hstart_combines['Total']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0


Merge House price index

In [152]:
df_copy = df.copy()

In [153]:
df['House price index'] = df_hpi_combine['Nominal Home Price Index']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582


Merge credit data, consumer loans

In [154]:
df_copy = df.copy()

In [155]:
df['Consumer loans'] = df_cloans['CONSUMER']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index,Consumer loans
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582,5.7786
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582,5.9030
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582,6.0272
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582,6.1559
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582,6.2822


Merge credit data, mortgage loans

In [156]:
df_copy = df.copy()

In [157]:
df['Mortgage loans'] = df_reloans['REALLN']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index,Consumer loans,Mortgage loans
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582,5.7786,8.8194
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582,5.9030,8.9518
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582,6.0272,9.0946
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582,6.1559,9.2382
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582,6.2822,9.3782


Merge Consumer price index

In [158]:
df_copy = df.copy()

In [159]:
df['CPI-U'] = df_cpiu_2['Inflation Index']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index,Consumer loans,Mortgage loans,CPI-U
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582,5.7786,8.8194,23.7
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582,5.9030,8.9518,23.5
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582,6.0272,9.0946,23.4
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582,6.1559,9.2382,23.8
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582,6.2822,9.3782,23.9


Merge inflation rates

In [160]:
df_copy = df.copy()

In [161]:
df['Inflation rate'] = df_inf_2['Inflation Index']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index,Consumer loans,Mortgage loans,CPI-U,Inflation rate
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582,5.7786,8.8194,23.7,0.10233
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582,5.9030,8.9518,23.5,0.09302
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582,6.0272,9.0946,23.4,0.06849
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582,6.1559,9.2382,23.8,0.08676
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582,6.2822,9.3782,23.9,0.09132


Merge Global Economic Growth data 

In [162]:
df_copy = df.copy()

In [163]:
df['World GDP growth'] = df_globa_2['GDP growth']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index,Consumer loans,Mortgage loans,CPI-U,Inflation rate,World GDP growth
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582,5.7786,8.8194,23.7,0.10233,0.0484
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582,5.9030,8.9518,23.5,0.09302,0.0484
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582,6.0272,9.0946,23.4,0.06849,0.0484
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582,6.1559,9.2382,23.8,0.08676,0.0484
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582,6.2822,9.3782,23.9,0.09132,0.0484


Merge S&P500

In [164]:
df_copy = df.copy()

In [165]:
df['SP500'] = df_snp500['Value']
df.head()

,Date,GDP,GDP rate,Recession dates,Unemployment,Ind.Prod. index,DJI,T-bill,Housing starts,House price index,Consumer loans,Mortgage loans,CPI-U,Inflation rate,World GDP growth,SP500
0,1948-01-01,2239.682,6.2,0,3.4,4.8654,176.26,0.97,1362.0,11.211582,5.7786,8.8194,23.7,0.10233,0.0484,14.83
1,1948-02-01,2239.682,6.2,0,3.8,4.6504,168.47,1.00,1362.0,11.211582,5.9030,8.9518,23.5,0.09302,0.0484,14.10
2,1948-03-01,2239.682,6.2,0,4.0,4.5160,169.94,1.00,1362.0,11.211582,6.0272,9.0946,23.4,0.06849,0.0484,14.30
3,1948-04-01,2276.690,6.8,0,3.9,4.5966,180.05,1.00,1362.0,11.211582,6.1559,9.2382,23.8,0.08676,0.0484,15.40
4,1948-05-01,2276.690,6.8,0,3.5,4.6235,186.38,1.00,1362.0,11.211582,6.2822,9.3782,23.9,0.09132,0.0484,16.15


In [166]:
col_list = df.columns.to_list()
dfo = df[['Date', 'GDP', 'GDP rate', 'Recession dates', 'Unemployment', 'Ind.Prod. index', 'DJI', 'SP500', 'T-bill', 'Housing starts', 'House price index', 'Consumer loans', 'Mortgage loans', 'CPI-U', 'Inflation rate', 'World GDP growth']]

In [167]:
df = dfo

In [168]:
df.size

14640

In [169]:
df.columns

Index(['Date', 'GDP', 'GDP rate', 'Recession dates', 'Unemployment',
       'Ind.Prod. index', 'DJI', 'SP500', 'T-bill', 'Housing starts',
       'House price index', 'Consumer loans', 'Mortgage loans', 'CPI-U',
       'Inflation rate', 'World GDP growth'],
      dtype='object')

Add FEDFUNDS, AAA_corb_bonds

In [170]:
df.to_csv('recession_dataset.csv', index=False)